# Lab 5

In [ ]:
-- Łączenie akcji I/O

-- Operator (>>) then -> Wykonuje ciąg akcji I/O
-- (>>) :: forall (m :: * -> *) a b. Monad m => m a -> m b -> m b
putChar 'a' >> putChar 'b' 

-- Operator (>>=) -> Przekierowuję z lewej strony na prawą dane (I/O)
-- getLine >>= putStrLn 
echo2 = getLine >>= \line -> putStrLn $ line ++ "!" 
echo3 :: IO () 
echo3 =  getLine >>= \l1 -> getLine >>= \l2 -> putStrLn $ l1 ++ l2 

-- Notacja do -> 
doActSeq = do 
  putChar 'A' 
  putChar 'G' 
  putChar 'H' 
  putChar '\n' 
doActSeq

doDialog = do
  putStr "What is your happy number? " 
  line <- getLine 
  let number = read line :: Int
    in 
      if number == 7 
                   then putStrLn "Ah, lucky 7!" 
                   else if odd number 
                        then putStrLn "Odd number! That's most people's choice..." 
                        else putStrLn "Hm, even number? Unusual!" 

In [ ]:
-- Operator fmap -> Wyciąga z kontekstu wartość, aplikuję funkcję, i zwraca
-- zapakowaną w ten sam kontekst wartość
-- fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b
fmap (+2) (Right 3) --Right 5
-- Left 3 -> is considered to be a failure value, so it is
-- immediately returned without further computations
fmap (+2) (Left 3) --Left 3
-- List functor
fmap (*2) [2] --[4]
fmap (+1) (*10) 1 --11
-- Interprets like left and right
fmap (+1) (0,0) --(0,1)

-- <$> -> same as fmap but in center
(+2) <$> (Right 3) --Right 5
(+1) <$> (Just 3) --Just 4
(\(x,y,z) -> (x+1,y+1)) <$> Just (0,0,0) --Just (1,1)

-- (<$) -> fmap but with ovverriding old value with given
1 <$ Right 2 --Right 1

In [ ]:
-- Dołączenie własnego typu do klasy Functor
data BinTree a = EmptyBT | NodeBT a (BinTree a) (BinTree a) deriving (Show)

instance Functor BinTree where
    fmap _ EmptyBT = EmptyBT
    fmap f (NodeBT a bl br) = NodeBT (f a) (fmap f bl) (fmap f br)

fmap (*2) EmptyBT
let leftTree = NodeBT 3 (EmptyBT) (EmptyBT)
let rightTree = NodeBT 4 (EmptyBT) (EmptyBT)
fmap (*2) (NodeBT 2 (leftTree) (rightTree))

data MyList a = EmptyList 
              | Cons a (MyList a) deriving Show 
 
instance Functor MyList where 
  fmap _ EmptyList    = EmptyList 
  fmap f (Cons x mxs) = Cons (f x) (fmap f mxs) 

In [ ]:
-- Funktory aplikatywne
-- <*> -> Stosuję zapakowaną w kontekst funkcję do zapakowanej w kontekst
-- wartości i zwraza zapakowaną nową wartość
Just (+2) <*> Just 3 --Just 5
[(*2), (+3)] <*> [1, 2, 3] --[2,4,6,4,5,6]
(*) <$> Just 5 <*> Just 3 --Just 15
-- Uwaga! f <$> x -> Zwraca zapakowaną funkcję, więc poniższe działa
(\x y z -> x+y+z) <$> Just 1 <*> Just 2 <*> Just 3 --Just 6
:t (<*>)

-- pure -> pakuję funkcję w kotekst
pure (\x y z -> x + y + z) <*> Just 1 <*> Just 2 <*> Just 3 
pure (+) <*> (Just 1) <*> (Just 2) 

-- (*>) -> Applicative f => f a -> f b -> f b
Just (+1) *> Just 5 --Just 5
Left 1 *> Left 2 --Left 1 (bo left odrazu jest zwracane)
(+1) *> (*100) $ 5 --500

-- (<*) -> f a -> f b -> f a 
Just 1 <* Just 5 -- Just 1
Left 1 <* Left 2  --Left 1 (bo left odrazu jest zwracane)
[1..2] <* [11..15] --[1,1,1,1,1,2,2,2,2,2] 
-- UWAGA (bo jest aplikowana dla każdej z wartości tablicy z prawej)
Just 2 <* Nothing  --Nothing
(+1) <* (*100) $ 5 --6

In [ ]:
-- Dołączenie własnego typu do klasy Applicative 
-- UWAGA! Functor powinien też być zdefioniowany
newtype Box a = MkBox a deriving Show 
 
instance Applicative Box where 
  pure = MkBox 
  (MkBox f) <*> w = fmap f w 
instance Functor Box where 
 fmap f (MkBox x) = MkBox (f x)

pure (*2) <*> MkBox 3 --MkBox 6
(*2) <$> MkBox 3 --MkBox 6
(+) <$> MkBox 1 <*> MkBox 2 --MkBox 3
(++) <$> MkBox "abc" <*> MkBox "def" --MkBox "abcdef"
(\x y z -> (z,y,x)) <$> 
    MkBox (Just 1) <*> 
        MkBox (Just 2) <*> 
            MkBox (Just 3) --MkBox (Just 3,Just 2,Just 1)

## Zadania Quizy Lab 5

In [ ]:
-- Zad 1. 
-- Przepisać powyższy kod z użyciem >> i >>=
fun = do
    putStrLn "Podaj imie: "
    s <- getLine
    putStr $ "Witaj " ++ s

-- putStrLn "Podaj imie: " >> getLine >>= \line ->putStr $ "Witaj "++ line

-- Zad 2.
-- Do czego ewaluuje się ??
-- foldr (+) 0 ( (*) <$> zipList [1,2,3]<*>((+1) ZipList [4,5,6]) )

-- Zad 3.
data Tree a = Node a (Tree a) (Tree a) | Leaf a deriving Show
instance Functor Tree where
    fmap f (Leaf a) = Leaf (f a)
    fmap f (Node a lt rt) = Node (f a) (fmap f lt) (fmap f rt) 
instance Applicative Tree where
    pure x = Node x (pure x) (pure x)
    Leaf f <*> a = fmap f a
    (Node a l r) <*> (Node a' l' r') = Node (a a') (l <*> l') (r <*> r') 
    
let functionsTree = Node (+1) (Leaf (+2)) (Leaf (+3))
let valuesTree = Node (1) (Leaf (2)) (Leaf (3))
functionsTree <*> valuesTree

-- Zad 4. 
-- Podaj wynik obliczenia poniższego wyrażenia
(filter odd) <$> Left [1..3]
-- Left [1,2,3]

-- Zad 5.
(\x y -> max <$> x <*> y) <$> [Just 1, Just 2] <*> [Just 3, Nothing]
-- [Just 3,Nothing,Just 3,Nothing]
max <$> [Just 1,Just 2] <*> [Just 3, Just 4]
-- There having matrix 
--     3   4
-- 1   (3) (4)
-- 2   (3) (4)
-- I odczytujemy wartosci w zwyklej kolejnosci

-- Zad 6. 
-- Podaj wynik obliczenia poniższego wyrażenia
(\x y z -> (x==y, z==y)) <$> Right 1 <*> Right 2 <*> Left 2
-- Left 2, bo to jest jakby błąd!

-- Zad 7.
-- Podaj wynik obliczenia poniższego wyrażenia
pure (<) <*> Right 1 <*> pure 3
-- Right True

-- Zad 8. 
-- Uzupełnij implementacje ??
-- data Either e a = Left e | Right a deriving Show
-- instance Functor (Either a) where
--     fmap f (Left e) = Left e
--     fmap f (Right a) = Right (f a)
-- fmap (\x -> 2*x ) Left 5

-- Zad 9. ??
-- data [] a = [] | a : [a] 

-- instance Applicative (Applicative a =>[a]) where
--     pure x = [x]
--     fs <*> xs = 
--         [f x | f<-fs, x<-xs]
    
-- Zad 10.
odd <$> [1..4]
-- [True,False,True,False]

-- Zad 11.
-- Przepisz poniższe akcję, wykorzystując notację do
getLine >>= \l1 -> return (l1 ++ l1) >>= \l2 -> print [l1,l2]

do
    l1<- getLine
    l2<-return (l1++l1)
    print [l1,l2]